### Imports

In [1]:
import os
import pandas as pd
import numpy as np

### Parameters

In [2]:
num_samples = 70 # quantidade de exemplos

### Inputs

In [3]:
edge_inputs = []
label_0_count = 0  
label_1_count = 0  
target_per_label = num_samples // 2

while len(edge_inputs) < num_samples:

    # Decidir qual tipo de aresta gerar para manter equilíbrio
    if label_0_count >= target_per_label:
        target_label = 1  # forçar aresta boa
    elif label_1_count >= target_per_label:
        target_label = 0  # forçar aresta ruim
    else:
        target_label = np.random.randint(0, 2)     # escolha aleatória
    
    # Gerar métricas baseadas no target_label
    if target_label == 1:  # aresta boa
        rssi = np.random.uniform(-65, -30)         # sinal mais forte
        etx = np.random.uniform(1, 4)              # menos retransmissões
        delay = np.random.uniform(1, 40)           # menor delay
        busy_fraction = np.random.uniform(0, 0.4)  # menos ocupado
    else:  # aresta ruim
        rssi = np.random.uniform(-90, -55)         # sinal mais fraco
        etx = np.random.uniform(3, 10)             # mais retransmissões
        delay = np.random.uniform(25, 100)         # maior delay
        busy_fraction = np.random.uniform(0.25, 1) # mais ocupado
    
    # Adicionar alguma variação para não ficar muito determinístico
    noise_factor = 0.15
    
    if np.random.random() < noise_factor:
        # Ocasionalmente criar uma exceção para adicionar realismo
        if target_label == 1:
            # Tornar algumas métricas piores
            if np.random.random() < 0.5:
                rssi = np.random.uniform(-80, -60)
            if np.random.random() < 0.5:
                etx = np.random.uniform(2, 6)
        else:
            # Tornar algumas métricas melhores
            if np.random.random() < 0.5:
                rssi = np.random.uniform(-70, -40)
            if np.random.random() < 0.5:
                delay = np.random.uniform(5, 50)
    
    # Contar labels para manter balanceamento
    if target_label == 0:
        label_0_count += 1
    else:
        label_1_count += 1
    
    # Arredondar valores para ficar mais realista
    rssi = round(rssi, 1)
    etx = round(etx, 1) 
    delay = round(delay)
    busy_fraction = round(busy_fraction, 2)
    
    edge_inputs.append([rssi, etx, delay, busy_fraction])

In [4]:
radnet_inputs = []
good_performance_count = 0  
poor_performance_count = 0  
target_per_performance = num_samples // 2

while len(radnet_inputs) < num_samples:
    # Decidir qual tipo de performance gerar para manter equilíbrio
    if good_performance_count >= target_per_performance:
        target_performance = 0  # forçar cenário de performance ruim
    elif poor_performance_count >= target_per_performance:
        target_performance = 1  # forçar cenário de boa performance
    else:
        target_performance = np.random.randint(0, 2)  # escolha aleatória
    
    # Gerar métricas baseadas no target_performance
    if target_performance == 1:  # cenário de boa performance
        # Baixa mobilidade, poucos pares comunicando, rede estável
        apmr_edge = np.random.uniform(0.75, 0.95)    # prefixes muito eficientes
        mfsr_edge = np.random.uniform(0.82, 0.95)    # alta taxa de sucesso
        idg_global = np.random.uniform(0.68, 0.90)   # boa entrega em grupo
    else:  # cenário de performance ruim
        # Alta mobilidade, muitos pares, rede congestionada
        apmr_edge = np.random.uniform(0.45, 0.72)    # prefixes menos eficientes
        mfsr_edge = np.random.uniform(0.50, 0.78)    # taxa de sucesso menor
        idg_global = np.random.uniform(0.45, 0.68)   # entrega mais difícil
    
    # Adicionar alguma variação para não ficar muito determinístico
    noise_factor = 0.15
    if np.random.random() < noise_factor:
        # Ocasionalmente criar uma exceção para adicionar realismo
        if target_performance == 1:
            # Tornar algumas métricas piores (interferência inesperada)
            if np.random.random() < 0.5:
                apmr_edge = np.random.uniform(0.65, 0.80)
            if np.random.random() < 0.5:
                mfsr_edge = np.random.uniform(0.75, 0.85)
        else:
            # Tornar algumas métricas melhores (condições favoráveis)
            if np.random.random() < 0.5:
                apmr_edge = np.random.uniform(0.70, 0.85)
            if np.random.random() < 0.5:
                idg_global = np.random.uniform(0.65, 0.80)
    
    # Contar performance para manter balanceamento
    if target_performance == 0:
        poor_performance_count += 1
    else:
        good_performance_count += 1
    
    # Garantir que os valores estejam nos ranges válidos [0,1]
    apmr_edge = max(0.45, min(0.98, apmr_edge))
    mfsr_edge = max(0.50, min(0.98, mfsr_edge))
    idg_global = max(0.45, min(0.95, idg_global))
    
    # Arredondar valores para ficar mais realista
    apmr_edge = round(apmr_edge, 6)
    mfsr_edge = round(mfsr_edge, 6)
    idg_global = round(idg_global, 2)
    
    radnet_inputs.append([apmr_edge, mfsr_edge, idg_global])

### Inputs Poor

In [13]:
edge_inputs = []

while len(edge_inputs) < num_samples:
    # Gerar sempre métricas ruins para edge inputs
    # Forçar pelo menos 2 métricas ruins por amostra
    
    # RSSI: sempre baixo (sinal fraco)
    rssi = np.random.uniform(-90, -60)  
    
    # ETX: tendência para alto (mais retransmissões)
    etx = np.random.uniform(4, 10)
    
    # Delay: tendência para alto
    delay = np.random.uniform(30, 100)
    
    # Busy fraction: tendência para alto (canal ocupado)
    busy_fraction = np.random.uniform(0.3, 1.0)
    
    # Adicionar algumas variações extremas
    extreme_factor = 0.3
    if np.random.random() < extreme_factor:
        if np.random.random() < 0.5:
            rssi = np.random.uniform(-95, -85)  # RSSI muito baixo
        if np.random.random() < 0.5:
            etx = np.random.uniform(7, 10)      # ETX muito alto
        if np.random.random() < 0.5:
            delay = np.random.uniform(70, 100)  # Delay muito alto
        if np.random.random() < 0.5:
            busy_fraction = np.random.uniform(0.8, 1.0)  # Canal muito ocupado
    
    # Arredondar valores
    rssi = round(rssi, 1)
    etx = round(etx, 1)
    delay = round(delay)
    busy_fraction = round(busy_fraction, 2)
    
    edge_inputs.append([rssi, etx, delay, busy_fraction])

In [ ]:
radnet_inputs = []

while len(radnet_inputs) < num_samples:
    # Gerar sempre métricas ruins para radnet
    # Cenários de alta mobilidade, muitos pares, rede congestionada
    
    # APMR: tendência baixa (prefixes ineficientes)
    apmr_edge = np.random.uniform(0.45, 0.65)
    
    # MFSR: tendência baixa (baixa taxa de sucesso)
    mfsr_edge = np.random.uniform(0.50, 0.70)
    
    # IDG: tendência baixa (entrega difícil em grupo)
    idg_global = np.random.uniform(0.45, 0.65)
    
    # Adicionar casos extremamente ruins
    extreme_factor = 0.25
    if np.random.random() < extreme_factor:
        if np.random.random() < 0.5:
            apmr_edge = np.random.uniform(0.45, 0.55)  # APMR muito baixo
        if np.random.random() < 0.5:
            mfsr_edge = np.random.uniform(0.50, 0.60)  # MFSR muito baixo
        if np.random.random() < 0.5:
            idg_global = np.random.uniform(0.45, 0.55)  # IDG muito baixo
    
    # Garantir ranges válidos
    apmr_edge = max(0.45, min(0.70, apmr_edge))
    mfsr_edge = max(0.50, min(0.75, mfsr_edge))
    idg_global = max(0.45, min(0.70, idg_global))
    
    # Arredondar valores
    apmr_edge = round(apmr_edge, 6)
    mfsr_edge = round(mfsr_edge, 6)
    idg_global = round(idg_global, 2)
    
    radnet_inputs.append([apmr_edge, mfsr_edge, idg_global])

### Inputs Goods

In [3]:
edge_inputs = []

while len(edge_inputs) < num_samples:
    # Gerar sempre métricas boas para edge inputs
    
    # RSSI: sempre alto (sinal forte)
    rssi = np.random.uniform(-65, -30)  
    
    # ETX: tendência para baixo (poucas retransmissões)
    etx = np.random.uniform(1, 4)
    
    # Delay: tendência para baixo
    delay = np.random.uniform(1, 40)
    
    # Busy fraction: tendência para baixo (canal livre)
    busy_fraction = np.random.uniform(0, 0.4)
    
    # Adicionar algumas variações extremas boas
    extreme_factor = 0.3
    if np.random.random() < extreme_factor:
        if np.random.random() < 0.5:
            rssi = np.random.uniform(-45, -30)  # RSSI muito alto
        if np.random.random() < 0.5:
            etx = np.random.uniform(1, 2)       # ETX muito baixo
        if np.random.random() < 0.5:
            delay = np.random.uniform(1, 20)    # Delay muito baixo
        if np.random.random() < 0.5:
            busy_fraction = np.random.uniform(0, 0.2)  # Canal muito livre
    
    # Arredondar valores
    rssi = round(rssi, 1)
    etx = round(etx, 1)
    delay = round(delay)
    busy_fraction = round(busy_fraction, 2)
    
    edge_inputs.append([rssi, etx, delay, busy_fraction])

In [14]:
radnet_inputs = []

while len(radnet_inputs) < num_samples:
    # Gerar sempre métricas boas para radnet
    # Cenários de baixa mobilidade, poucos pares, rede estável
    
    # APMR: tendência alta (prefixes eficientes)
    apmr_edge = np.random.uniform(0.75, 0.95)
    
    # MFSR: tendência alta (alta taxa de sucesso)
    mfsr_edge = np.random.uniform(0.80, 0.95)
    
    # IDG: tendência alta (entrega eficiente em grupo)
    idg_global = np.random.uniform(0.70, 0.90)
    
    # Adicionar casos extremamente bons
    extreme_factor = 0.25
    if np.random.random() < extreme_factor:
        if np.random.random() < 0.5:
            apmr_edge = np.random.uniform(0.85, 0.95)  # APMR muito alto
        if np.random.random() < 0.5:
            mfsr_edge = np.random.uniform(0.88, 0.95)  # MFSR muito alto
        if np.random.random() < 0.5:
            idg_global = np.random.uniform(0.80, 0.90)  # IDG muito alto
    
    # Garantir ranges válidos
    apmr_edge = max(0.70, min(0.98, apmr_edge))
    mfsr_edge = max(0.75, min(0.98, mfsr_edge))
    idg_global = max(0.65, min(0.95, idg_global))
    
    # Arredondar valores
    apmr_edge = round(apmr_edge, 6)
    mfsr_edge = round(mfsr_edge, 6)
    idg_global = round(idg_global, 2)
    
    radnet_inputs.append([apmr_edge, mfsr_edge, idg_global])

### Save

In [5]:
edges_input = pd.DataFrame(edge_inputs, columns=["rssi", "etx", "delay", "busy_fraction"])
os.makedirs("../inputs", exist_ok=True)
edges_input.to_csv("../inputs/edges_input.csv", index=False)

print("Edges Inputs salvo em ../datasets/edges_input.csv")

Edges Inputs salvo em ../datasets/edges_input.csv


In [6]:
radnet_input = pd.DataFrame(radnet_inputs, columns=["apmr_edge", "mfsr_edge", "idg_global"])
os.makedirs("../inputs", exist_ok=True)
radnet_input.to_csv("../inputs/radnet_input.csv", index=False)

print("Radnet Input salvo em ../datasets/radnet_input.csv")

Radnet Input salvo em ../datasets/radnet_input.csv
